# DIET PROBLEM - PYOMO

*Zuria Bauer Hartwig* ( [CAChemE](http://cacheme.org))

Original Problem: [Linear and Integer Programming](https://www.coursera.org/course/linearprogramming) (Coursera Course) - University of Colorado Boulder & University of Colorado System

Based on the Examples from the Optimization Course = [Taller-Optimizacion-Python-Pyomo](https://github.com/CAChemE/Taller-Optimizacion-Python-Pyomo) from [CAChemE.org](http://cacheme.org/optimizacion-programacion-matematica-con-python-pyomo/)

[Neos Guide - The Diet Problem](http://www.neos-guide.org/content/diet-problem)

### SUMMARY

The goal of the diet problem is to select a set of foods we can get at McDonaldsx,
that will satisfy a set of daily nutritional requirement at minimum cost. 
The problem is formulated as a linear program where the objective is to minimize cost 
and the constraints are to satisfy the specified nutritional requirements. 
The diet problem constraints typically regulate the number of calories,
the calories from Fat, Total Fat, Cholesterol, Sodium, 
Carbohydrates, Fiber, Sugar, Proteine, Vitamine A and C,
Calcium and Sodium in the diet. 
While the mathematical formulation is simple, 
the solution may not be palatable! 
The nutritional requirements can be met without regard for taste or variety, 
so consider the output before digging into a meal from an "optimal" menu!


#### 1. Import

In [52]:
# Loading pyomo
from pyomo.environ import *
infinity = float('inf')

#### 2. Data

Let's get the data:

- http://nutrition.mcdonalds.com/getnutrition/nutritionfacts.pdf
- http://www.fastfoodmenuprices.com/mcdonalds-prices/



In [53]:
#Data
!cat diet.dat

#To show the data we can use !cat (or !type for windows) commands

# http://nutrition.mcdonalds.com/getnutrition/nutritionfacts.pdf
# http://www.fastfoodmenuprices.com/mcdonalds-prices/

#Food + Cost

param:  F:                                                  c  :=
  "Bacon Clubhouse Burgerx"                                 4.49
  "Bacon Clubhouse Grilled Chicken Sandwichx"               4.00
  "Bacon Clubhouse Crispy Chicken Sandwichx"                4.00
  "McChickenx"                                              4.00
  "Buffalo Ranch McChickenx"                                1.00
  "Bacon Buffalo Ranch McChickenx"                          1.00
  "Southern Style Buttermilk Crispy Chicken Sandwichx"      2.00
  "Filet-O-Fishx"                                           4.79
  "McRibx"                                                  3.79
  "Premium McWrap Chicken & Bacon (Crispy)x"                2.99
  "Premium McWrap Chicken & Bacon (Grilled)x"               3.99
  "Premium McWrap Chicken & Ranch (Crispy)x"                3.99               
  "Pr

### Solving

#### 3. Model

In [54]:
# Creation of a Concrete Model
model = AbstractModel()


#### 4. Sets

In [55]:
#DEFINE SETS

# Products
model.F = Set()
# Nutrients
model.N = Set()

#### 5. Parameters

In [56]:
# DEFINE PARAMETERS

# Cost
model.c = Param(model.F, within = PositiveReals)

# Amount of nutrient
model.a    = Param(model.F, model.N, within = NonNegativeReals)

# Max and Min for each Nutrient
model.Nmin = Param(model.N, within = NonNegativeReals, default = 0.0)
model.Nmax = Param(model.N, within = NonNegativeReals, default = infinity)


#### 6. Variables

In [57]:
# Number of servings
model.x = Var(model.F, within = NonNegativeIntegers)


#### 7. Objective

In [58]:
# Minimize z(cost)
def cost(model):
    return sum(model.c[i] * model.x[i] for i in model.F)
model.cost = Objective(rule=cost)


#### 8. Constrains

In [59]:
#CONSTRAINS

#Max
def nutrients_max(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return value <= model.Nmax[j]
model.nutrient_limit_max = Constraint(model.N, rule=nutrients_max)
#Min
def nutrient_min(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return model.Nmin[j] <= value 
model.nutrient_limit_min = Constraint(model.N, rule=nutrient_min)

#### 9. Solution

In [60]:
#Get our Solution:

!pyomo solve --solver=glpk code.py diet.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.06] Applying solver
[    0.09] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 2.0
    Solver results file: results.yml
[    0.09] Applying Pyomo postprocessing actions
[    0.09] Pyomo Finished


#### 10. Results

In [61]:
#Results
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2.0
  Upper bound: 2.0
  Number of objectives: 1
  Number of constraints: 22
  Number of variables: 43
  Number of nonzeros: 869
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 19
      Number of created subproblems: 19
  Error rc: 0
  Time: 0.011977434158325195
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------